In [1]:
import datetime
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

                                
class Report():
    
    class _Dic2obj():
        def __init__(self, dic):
            for key in dic:
                value = dic[key]
                if type(value) == dict:
                    obj = Report._Dic2obj(value)
                    setattr(self, key, obj)
                else:
                    setattr(self, key, value)
    
    def __init__(self, db_path):
        
        self.source_path = os.path.join(db_path, "source")
        with open(os.path.join(db_path, "reference.json")) as f:
            self.ref = self._Dic2obj(json.load(f))
        with open(os.path.join(db_path, "coin_list.json")) as f:
            coins = json.load(f)
        with open(os.path.join(db_path, "template.json")) as f:
            template = json.load(f)
        
        self.current_time = (datetime.datetime.now()
                             .strftime("%Y-%m-%d %H:%M:%S"))
        self.coins = sorted(list(
            set(sum([coin.split("-") for coin in coins], []))
            -{"BTRX", "USDT"}
        ))
        self.report = {coin: template.copy() for coin in self.coins}
        
        self.source = {}
        filepath = os.path.join(db_path, "source")
        self.prev_source = (pd.read_pickle(filepath)
                            if os.path.isfile(filepath)
                            else {})
    
    def set_text(self, url):
        response = (self.prev_source[url]
                    if url in self.prev_source
                    else requests.get(url))
        self.source.update({url: response})

    def get_text(self, url):
        return self.source[url].text
    
    def save_source(self):
        pd.to_pickle(self.source, self.source_path)
    
    def fill_numbers(self):
        
        coins, source, report = self.coins, self.source, self.report
        
        site = self.ref.coinmarketcap
        self.set_text(site.quick_search)
        quick_search = json.loads(self.get_text(site.quick_search))
        
        blacklist = ["batcoin"]
        self.quick_search = list(filter(
            lambda dic: dic["slug"] not in blacklist,
            quick_search
        ))
        
        self.name_map = {dic["symbol"]: dic["slug"]
                         for dic in self.quick_search}
        self.urls = {coin: site.currencies+self.name_map[coin]
                     for coin in coins}
        for coin in coins:
            self.set_text(self.urls[coin])
            
        self.save_source()        
        
        urls = self.urls
        for coin in coins:
            text = self.get_text(urls[coin])
            bs = BeautifulSoup(text, "lxml")
            selector = "span[data-currency-volume]"
            value = bs.select_one(selector)["data-usd"]
            report[coin]["Daily liquidity"] = float(value)
            selector = "span[data-currency-market-cap]"
            value = bs.select_one(selector)["data-usd"]
            report[coin]["Market cap"] = float(value)

In [2]:
report = Report("./db")
report.fill_numbers()

In [3]:
index = 0
coin = list(report.report.keys())[index]
print(report.urls[coin])

report.report[coin]["Summary"] = """
"""
report.report[coin]["Base algorithm"] = ""
report.report[coin]["Mining"] = ""

https://coinmarketcap.com/currencies/cardano
